In [1]:
import pandas as pd
import os
import ast
# import sys
# sys.path.append("D:\PUBLIC_DATA_PROJECT\Domain_cluster")

from tools.crawl import WebScraper 
from tools.preprocessing import preprocessing

In [2]:
#Config
storage_folder = "./crawl/company_information"
os.makedirs(storage_folder, exist_ok=True)

path_data = "./company_1.csv"

storage_folder_contain_subdomain = "./crawl/company_information_subdomain"
os.makedirs(storage_folder_contain_subdomain, exist_ok=True)

# DATA

In [3]:
# Company profiles

df = pd.read_csv(path_data)
# df1 = pd.read_csv("D:\PUBLIC_DATA_PROJECT\Check_main_url\info_company1.csv")
# df = pd.concat([df, df1], axis=0).reset_index(drop=True)
df["code"] = df["code"].astype(str)
df.head()

,link_company,name_company,code,status_of_company,list_business,list_website,Status
0,https://hosocongty.vn/tap-doan-vingroup-cong-t...,TẬP ĐOÀN VINGROUP - CÔNG TY CP,101245486.0,Đang hoạt động (đã được cấp GCN ĐKT),"[{'5510': 'Dịch vụ lưu trú ngắn ngày'}, {'4772...","[{'title': 'Trang chủ - Tập đoàn Vingroup', 'u...",True
1,https://hosocongty.vn/tong-cong-ty-bao-hiem-ba...,TỔNG CÔNG TY BẢO HIỂM BẢO VIỆT,101527385.0,NaN,[{'65129': 'Bảo hiểm phi nhân thọ khác'}],"[{'title': 'Tổng Công ty Bảo hiểm Bảo Việt', '...",True
2,https://hosocongty.vn/cong-ty-co-phan-bibica-c...,CÔNG TY CỔ PHẦN BIBICA,3600363970.0,Đang hoạt động (đã được cấp GCN ĐKT),"[{'1071': 'Sản xuất các loại bánh từ bột'}, {'...","[{'title': 'Bibica - Tinh hoa bánh kẹo Việt', ...",True
3,https://hosocongty.vn/cong-ty-co-phan-bao-hiem...,CÔNG TY CỔ PHẦN BẢO HIỂM HÀNG KHÔNG,102737963.0,NaN,[{'64': 'Hoạt động dịch vụ tài chính (trừ bảo ...,[{'title': 'VNI - Tổng Công Ty Cổ Phần Bảo hiể...,True
4,https://hosocongty.vn/cong-ty-co-phan-nhua-va-...,CÔNG TY CỔ PHẦN NHỰA VÀ MÔI TRƯỜNG XANH AN PHÁT,800789746.0,NaN,[{'963': 'Hoạt động dịch vụ phục vụ cá nhân kh...,"[{'title': 'An Phat Bioplastics', 'url': 'http...",True


In [4]:
'''Input requirements
    Schema: 'link_company', 'name_company', 'code', 'status_of_company',
       'list_business', 'list_website', 'Status', 'international_name',
       'acronyms_name', 'address', 'representative', 'phone_number',
       'day_of_operation', 'main_job', 'update_time', 'id'
''' 

df = preprocessing(df)

In [5]:
# SCRAPING DATA
'''
    saving_path: Storage location for crawled data
    get_subdomain: to False to crawl only one web text per URL
                   to True to crawl 5 sub-domain web-text and main web per URL
'''


Worker = WebScraper(saving_path=storage_folder,
                    get_subdomain=False)

# Hàm để crawl URL từ cột list_business
def task(row):
    print(f"Company: {row['name_company']}")
    print(f"Tax Code: {row['code']}")
    tax = row['code'].split(".")[0]

    business_list = row['extracted_urls']

    # Duyệt qua từng URL và in ra
    for index, business in enumerate(business_list):
        # url = business['url']
        url = business
        try:
            Worker.process_url(url, tax, index)
        except:
            continue

    print("="*50)


for i in range(len(df)):
    row = df.iloc[i, :]
    task(row)

Company: TẬP ĐOÀN VINGROUP -  CÔNG TY CP
Tax Code: 101245486.0
url:  https://vingroup.net/
url: https://vingroup.net/ 
 Saved: ./crawl/company_information/company_information/101245486/0.txt
url:  https://mekongasean.vn/to-ng-ta-i-sa-n-vingroup-vuot-nguong-700000-ty-do-ng-31407.html#:~:text=T%E1%BB%95ng%20s%E1%BB%91%20n%E1%BB%A3%20c%E1%BB%A7a%20Vingroup,l%E1%BB%9Bn%20v%E1%BB%9Bi%20418.112%20t%E1%BB%B7%20%C4%91%E1%BB%93ng.
url: https://mekongasean.vn/to-ng-ta-i-sa-n-vingroup-vuot-nguong-700000-ty-do-ng-31407.html#:~:text=T%E1%BB%95ng%20s%E1%BB%91%20n%E1%BB%A3%20c%E1%BB%A7a%20Vingroup,l%E1%BB%9Bn%20v%E1%BB%9Bi%20418.112%20t%E1%BB%B7%20%C4%91%E1%BB%93ng. 
 Saved: ./crawl/company_information/company_information/101245486/1.txt
Company: TỔNG CÔNG TY BẢO HIỂM BẢO VIỆT
Tax Code: 101527385.0
url:  https://www.baoviet.com.vn/insurance/
url: https://www.baoviet.com.vn/insurance/ 
 Saved: ./crawl/company_information/company_information/101527385/0.txt
url:  https://baovietonline.com.vn/
url: http

In [6]:
# BUILDING META DATA
business_meta = {}

# Hàm để in URL từ cột list_business
def task(row, business):
    print(f"Company: {row['name_company']}")
    print(f"Tax Code: {row['code']}")
    tax = row['code'].split(".")[0]
    business_list = row['extracted_urls']
    # Duyệt qua từng URL và in ra

    urls = []
    paths = []
    for index, business in enumerate(business_list):
        # url = business['url']
        url = business
        path = f"{storage_folder}/{tax}/{index}.txt"

        if os.path.exists(path):
            urls.append(url)
            paths.append(path)

            industry_list = ["".join(x.values()) for x in ast.literal_eval(row["list_business"])]
            industry_list = ",".join(industry_list)


    if len(paths) > 0:
        business_meta[tax] = {
            'urls': urls,
            'paths': paths,
            'name': row['name_company'],
            'industry_text': industry_list
        }
        print(business)

    print("="*50)


for i in range(len(df)):
    row = df.iloc[i, :]
    task(row, business_meta)

Company: TẬP ĐOÀN VINGROUP -  CÔNG TY CP
Tax Code: 101245486.0
https://mekongasean.vn/to-ng-ta-i-sa-n-vingroup-vuot-nguong-700000-ty-do-ng-31407.html#:~:text=T%E1%BB%95ng%20s%E1%BB%91%20n%E1%BB%A3%20c%E1%BB%A7a%20Vingroup,l%E1%BB%9Bn%20v%E1%BB%9Bi%20418.112%20t%E1%BB%B7%20%C4%91%E1%BB%93ng.
Company: TỔNG CÔNG TY BẢO HIỂM BẢO VIỆT
Tax Code: 101527385.0
https://baoviet.com.vn/
Company: CÔNG TY CỔ PHẦN BIBICA
Tax Code: 3600363970.0
https://finance.vietstock.vn/bbc/ho-so-doanh-nghiep.htm
Company: CÔNG TY CỔ PHẦN BẢO HIỂM HÀNG KHÔNG
Tax Code: 102737963.0
https://ibaohiem.vn/gioi-thieu-bao-hiem-hang-khong-vni/
Company: CÔNG TY CỔ PHẦN NHỰA VÀ MÔI TRƯỜNG XANH AN PHÁT
Tax Code: 800789746.0
https://www.fact-link.com.vn/home/anphat-vn


In [16]:
business_meta

{'101245486': {'urls': ['https://vingroup.net/',
   'https://mekongasean.vn/to-ng-ta-i-sa-n-vingroup-vuot-nguong-700000-ty-do-ng-31407.html#:~:text=T%E1%BB%95ng%20s%E1%BB%91%20n%E1%BB%A3%20c%E1%BB%A7a%20Vingroup,l%E1%BB%9Bn%20v%E1%BB%9Bi%20418.112%20t%E1%BB%B7%20%C4%91%E1%BB%93ng.'],
  'paths': ['./crawl/company_information/101245486/0.txt',
   './crawl/company_information/101245486/1.txt'],
  'name': 'TẬP ĐOÀN VINGROUP -  CÔNG TY CP',
  'industry_text': 'Dịch vụ lưu trú ngắn ngày,Bán lẻ thuốc, dụng cụ y tế, mỹ phẩm và vật phẩm vệ sinh trong các cửa hàng chuyên doanh,Nhà hàng và các dịch vụ ăn uống phục vụ lưu động,Tư vấn máy vi tính và quản trị hệ thống máy vi tính,Hoạt động dịch vụ công nghệ thông tin và dịch vụ khác liên quan đến máy vi tính,Xử lý dữ liệu, cho thuê và các hoạt động liên quan,Khai thác, xử lý và cung cấp nước,Thoát nước và xử lý nước thải,Thu gom rác thải không độc hại,Phá dỡ,Chuẩn bị mặt bằng,Lắp đặt hệ thống điện,Lắp đặt hệ thống xây dựng khác,Hoàn thiện công trình

# MODEL

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModel
import torch
from underthesea import word_tokenize


class SimilarityCalculator:
    def __init__(self):
        # Load models
        self.sbert_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
        self.labse_model = SentenceTransformer('sentence-transformers/LaBSE')
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
        # self.phobert_model = AutoModel.from_pretrained("vinai/phobert-base")
        self.vectorizer = TfidfVectorizer(stop_words='english')  # Can be customized for Vietnamese stop words

    def preprocess_text(self, text):
        # Tokenize Vietnamese text
        return ' '.join(word_tokenize(text))

    def get_embedding_sbert(self, text):
        return self.sbert_model.encode(text)

    def get_embedding_labse(self, text):
        return self.labse_model.encode(text)

    def get_embedding_phobert(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=2000)
        with torch.no_grad():
            outputs = self.phobert_model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        return cls_embedding

    def get_similarity(self, urls, industry_text, website_texts, model_name='all'):
        # Preprocess the text
        industry_text = self.preprocess_text(industry_text)
        website_texts = [self.preprocess_text(text) for text in website_texts]

        # Initialize DataFrame to store results
        results = pd.DataFrame({'url': [urls[i] for i in range(len(website_texts))]})

        if model_name in ['all', 'sbert']:
            industry_embedding_sbert = self.get_embedding_sbert(industry_text)
            website_embeddings_sbert = [self.get_embedding_sbert(text) for text in website_texts]
            similarities_sbert = cosine_similarity([industry_embedding_sbert], website_embeddings_sbert)[0]
            results['SBERT Similarity'] = similarities_sbert

        if model_name in ['all', 'labse']:
            industry_embedding_labse = self.get_embedding_labse(industry_text)
            website_embeddings_labse = [self.get_embedding_labse(text) for text in website_texts]
            similarities_labse = cosine_similarity([industry_embedding_labse], website_embeddings_labse)[0]
            results['LaBSE Similarity'] = similarities_labse

        if model_name in ['all', 'tfidf']:
            all_texts = [industry_text] + website_texts
            tfidf_matrix = self.vectorizer.fit_transform(all_texts)
            industry_vector_tfidf = tfidf_matrix[0]
            website_vectors_tfidf = tfidf_matrix[1:]
            similarities_tfidf = cosine_similarity(industry_vector_tfidf, website_vectors_tfidf)[0]
            results['TF-IDF Similarity'] = similarities_tfidf

        # if model_name in ['all', 'phobert']:
        #     industry_embedding_phobert = self.get_embedding_phobert(industry_text)
        #     website_embeddings_phobert = [self.get_embedding_phobert(text) for text in website_texts]
        #     similarities_phobert = cosine_similarity([industry_embedding_phobert], website_embeddings_phobert)[0]
        #     results['PhoBERT Similarity'] = similarities_phobert
        
        return results



d:\DATN\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
d:\DATN\.venv\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
d:\DATN\.venv\lib\site-packages\torchvision\transforms\v2\__init__.py:54

In [8]:
# Example usage
industry_text = "Nganh, nghe )( buon ban du lieu"
website_texts = [
    "Chung toi buon ban )( du lieu va cong nghe",
    "Ben cung cap cong nghe",
    "Hien tai chung toi dang phat trien he thong",
]
websites = ["web1", "web2", "web3"]
# Initialize the calculator
similarity_calculator = SimilarityCalculator()

# Get similarities with all models
results = similarity_calculator.get_similarity(websites,industry_text, website_texts, model_name='labse')
print(results)

    url  LaBSE Similarity
0  web1          0.782316
1  web2          0.443528
2  web3          0.255703


# INFERENCE

In [31]:
def load_text_from_paths(paths):
    # Initialize an empty list to store website texts
    website_texts = []
    
    # Read each file's content
    for path in paths:
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read().strip()  # Remove any extra whitespace
            website_texts.append(content)
    
    return website_texts



In [32]:
taxs_code = list(business_meta.keys())
similarity_calculator = SimilarityCalculator()

results = pd.DataFrame([])
for tax_code in taxs_code:
    data = business_meta[tax_code]
    # Load texts from paths
    website_texts = load_text_from_paths(data['paths'])

    # Print output
    industry_text = data['industry_text']

    # Get similarities with all models
    for url, text in zip(data["urls"], website_texts):
        print(f"url: {url}")
        result = similarity_calculator.get_similarity([url] ,industry_text, [text], model_name='labse')
        torch.cuda.empty_cache()

        result["name"] = [data['name']]*len(result)
        result["code"] = [tax_code]*len(result)
        result["industry_text"] = [data['industry_text']]*len(result)

        results = pd.concat([results, result], axis=0)

    torch.cuda.empty_cache()

url: https://vingroup.net/
url: https://mekongasean.vn/to-ng-ta-i-sa-n-vingroup-vuot-nguong-700000-ty-do-ng-31407.html#:~:text=T%E1%BB%95ng%20s%E1%BB%91%20n%E1%BB%A3%20c%E1%BB%A7a%20Vingroup,l%E1%BB%9Bn%20v%E1%BB%9Bi%20418.112%20t%E1%BB%B7%20%C4%91%E1%BB%93ng.
url: https://www.baoviet.com.vn/insurance/
url: https://baovietonline.com.vn/
url: https://baoviet.com.vn/
url: https://www.bibica.com.vn/
url: https://s.cafef.vn/hose/BBC-cong-ty-co-phan-bibica.chn
url: https://finance.vietstock.vn/bbc/ho-so-doanh-nghiep.htm
url: https://bhhk.com.vn/
url: https://ebhhk.com.vn/gioi-thieu.html
url: https://ibaohiem.vn/gioi-thieu-bao-hiem-hang-khong-vni/
url: https://anphatbioplastics.com/
url: https://finance.vietstock.vn/AAA-ctcp-nhua-an-phat-xanh.htm
url: https://www.fact-link.com.vn/home/anphat-vn


In [37]:
results = results.reset_index(drop=True)

In [39]:
# Lấy các hàng có similarity cao nhất trong mỗi nhóm name
highest_similarity_df = results.loc[results.groupby("code")["LaBSE Similarity"].idxmax()].reset_index(drop=True)
highest_similarity_df

,url,LaBSE Similarity,name,code,industry_text
0,https://vingroup.net/,0.567719,TẬP ĐOÀN VINGROUP - CÔNG TY CP,101245486,"Dịch vụ lưu trú ngắn ngày,Bán lẻ thuốc, dụng c..."
1,https://baovietonline.com.vn/,0.342600,TỔNG CÔNG TY BẢO HIỂM BẢO VIỆT,101527385,Bảo hiểm phi nhân thọ khác
2,https://ibaohiem.vn/gioi-thieu-bao-hiem-hang-k...,0.455859,CÔNG TY CỔ PHẦN BẢO HIỂM HÀNG KHÔNG,102737963,Hoạt động dịch vụ tài chính (trừ bảo hiểm và b...
3,https://www.bibica.com.vn/,0.516349,CÔNG TY CỔ PHẦN BIBICA,3600363970,"Sản xuất các loại bánh từ bột,Sản xuất giấy nh..."
4,https://anphatbioplastics.com/,0.177271,CÔNG TY CỔ PHẦN NHỰA VÀ MÔI TRƯỜNG XANH AN PHÁT,800789746,Hoạt động dịch vụ phục vụ cá nhân khác chưa đư...


In [40]:
highest_similarity_df.to_csv("./business_meta.csv")

# Scraping 5 sub-domain


In [3]:
df_bank = pd.DataFrame({"url": ["https://www.vietcombank.com.vn/"], 
                        "code": ["123"]})
meta_df = pd.read_csv("./business_meta.csv")
meta_df = pd.concat([df_bank, meta_df], axis=0)
meta_df

,url,code,Unnamed: 0,LaBSE Similarity,name,industry_text
0,https://www.vietcombank.com.vn/,123,NaN,NaN,NaN,NaN
0,https://vingroup.net/,101245486,0.0,0.567719,TẬP ĐOÀN VINGROUP - CÔNG TY CP,"Dịch vụ lưu trú ngắn ngày,Bán lẻ thuốc, dụng c..."
1,https://baovietonline.com.vn/,101527385,1.0,0.342600,TỔNG CÔNG TY BẢO HIỂM BẢO VIỆT,Bảo hiểm phi nhân thọ khác
2,https://ibaohiem.vn/gioi-thieu-bao-hiem-hang-k...,102737963,2.0,0.455859,CÔNG TY CỔ PHẦN BẢO HIỂM HÀNG KHÔNG,Hoạt động dịch vụ tài chính (trừ bảo hiểm và b...
3,https://www.bibica.com.vn/,3600363970,3.0,0.516349,CÔNG TY CỔ PHẦN BIBICA,"Sản xuất các loại bánh từ bột,Sản xuất giấy nh..."
4,https://anphatbioplastics.com/,800789746,4.0,0.177271,CÔNG TY CỔ PHẦN NHỰA VÀ MÔI TRƯỜNG XANH AN PHÁT,Hoạt động dịch vụ phục vụ cá nhân khác chưa đư...


In [ ]:
# SCRAPING DATA
'''
    saving_path: Storage location for crawled data
    get_subdomain: to False to crawl only one web text per URL
                   to True to crawl 5 sub-domain web-text and main web per URL
'''


Worker = WebScraper(saving_path=storage_folder_contain_subdomain,
                    get_subdomain=True)

# Hàm để crawl URL từ cột list_business
def task(row):
    print(f"Company: {row['name']}")
    print(f"Tax Code: {row['code']}")
    tax = row['code']
    url = row['url']

    try:
        Worker.process_url_subdomain(url, tax)
    except Exception as e:
        print(e)

    print("="*50)


for i in range(len(meta_df)):
    row = meta_df.iloc[i, :]
    task(row)

Company: nan
Tax Code: 123
url:  https://www.vietcombank.com.vn/
sub_domains in url:  0
url: https://www.vietcombank.com.vn/ 
 Saved: ./crawl/company_information_subdomain/123/0.txt
Company: TẬP ĐOÀN VINGROUP -  CÔNG TY CP
Tax Code: 101245486
url:  https://vingroup.net/
sub_domains in url:  1
url: https://vingroup.net/ 
 Saved: ./crawl/company_information_subdomain/101245486/0.txt
Company: TỔNG CÔNG TY BẢO HIỂM BẢO VIỆT
Tax Code: 101527385
url:  https://baovietonline.com.vn/
sub_domains in url:  92
url: https://baovietonline.com.vn/ 
 Saved: ./crawl/company_information_subdomain/101527385/0.txt
url: https://baovietonline.com.vn/vi/dich-vu-chi-tra-boi-thuong.html 
 Saved: ./crawl/company_information_subdomain/101527385/1.txt
url: https://baovietonline.com.vn/vi/gio-hang.html 
 Saved: ./crawl/company_information_subdomain/101527385/2.txt
url: https://baovietonline.com.vn/vi/bao-hiem-ca-nhan/4/bao-hiem-10-benh-hiem-ngheo.html 
 Saved: ./crawl/company_information_subdomain/101527385/3.txt


In [2]:
!google-chrome --version


'google-chrome' is not recognized as an internal or external command,
operable program or batch file.


# Multi Threading